In [25]:
import pandas as pd
import os
import numpy as np
import torch

informations_households.csv : this file that contains all the information on the households in the panel (their acorn group, their tariff) and in which block.csv.gz file their data are stored

halfhourly_dataset.zip: Zip file that contains the block files with the half-hourly smart meter measurement

daily_dataset.zip: Zip file that contains the block files with the daily information like the number of measures, minimum, maximum, mean, median, sum and std.

acorn_details.csv : Details on the acorn groups and their profile of the people in the group, it's come from this xlsx spreadsheet.The first three columns are the attributes studied, the ACORN-X is the index of the attribute. At a national scale, the index is 100 if for one column the value is 150 it means that there are 1.5 times more people with this attribute in the ACORN group than at the national scale. You can find an explanation on the CACI website

weather_daily_darksky.csv : that contains the daily data from darksky api. You can find more details about the parameters in the documentation of the api

weather_hourly_darksky.csv : that contains the hourly data from darksky api. You can find more details about the parameters in the documentation of the api



In [26]:

for dirname, _, filenames in os.walk('./data/London'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/London/informations_households.csv
./data/London/daily_dataset.csv
./data/London/weather_daily_darksky.csv
./data/London/energy_block40.pt
./data/London/acorn_details.csv
./data/London/darksky_parameters_documentation.html
./data/London/uk_bank_holidays.csv
./data/London/weather_hourly_darksky.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_59.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_20.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_4.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_73.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_30.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_29.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_58.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_76.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_47.csv
./data/London/halfhourly_dataset/halfhourly_dataset/block_56.csv
./data/London/halfhourly_dataset/halfhourly_d

# Plot function

# informations_households

In [27]:
informations_households = pd.read_csv('./data/London/informations_households.csv', delimiter=',')
informations_households.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0
2,MAC000002,Std,ACORN-A,Affluent,block_0
3,MAC003613,Std,ACORN-A,Affluent,block_0
4,MAC003597,Std,ACORN-A,Affluent,block_0


# Acron details

# UK Bank Holidays

In [30]:
uk_bank_holidays = pd.read_csv('./data/London/uk_bank_holidays.csv')
uk_bank_holidays.head(10)

,Bank holidays,Type
0,2012-12-26,Boxing Day
1,2012-12-25,Christmas Day
2,2012-08-27,Summer bank holiday
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday)
4,2012-04-06,Spring bank holiday (substitute day)
5,2012-07-05,Early May bank holiday
6,2012-09-04,Easter Monday
7,2012-06-04,Good Friday
8,2012-02-01,New Year?s Day (substitute day)
9,2013-12-26,Boxing Day


# Weather


In [31]:
weather = pd.read_csv("./data/London/weather_hourly_darksky.csv")
weather.head()

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2,3.70,98,9.46,2011-11-11 02:00:00,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
3,3.12,99,9.23,2011-11-11 03:00:00,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
4,1.85,111,9.26,2011-11-11 04:00:00,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy


# Half hourly dataset

In [54]:
block= 'block_0'
load = pd.read_csv('./data/London/halfhourly_dataset/halfhourly_dataset/{}.csv'.format(block))
load

,LCLid,tstp,energy(kWh/hh)
0,MAC000002,2012-10-12 00:30:00.0000000,0
1,MAC000002,2012-10-12 01:00:00.0000000,0
2,MAC000002,2012-10-12 01:30:00.0000000,0
3,MAC000002,2012-10-12 02:00:00.0000000,0
4,MAC000002,2012-10-12 02:30:00.0000000,0
...,...,...,...
1222665,MAC005492,2014-02-27 22:00:00.0000000,0.182
1222666,MAC005492,2014-02-27 22:30:00.0000000,0.122
1222667,MAC005492,2014-02-27 23:00:00.0000000,0.14
1222668,MAC005492,2014-02-27 23:30:00.0000000,0.192


In [55]:
load['tstp'] = pd.to_datetime(load['tstp'])
# Extract date and time
load['date'] = load['tstp'].dt.date
load['time'] = load['tstp'].dt.time
load

,LCLid,tstp,energy(kWh/hh),date,time
0,MAC000002,2012-10-12 00:30:00,0,2012-10-12,00:30:00
1,MAC000002,2012-10-12 01:00:00,0,2012-10-12,01:00:00
2,MAC000002,2012-10-12 01:30:00,0,2012-10-12,01:30:00
3,MAC000002,2012-10-12 02:00:00,0,2012-10-12,02:00:00
4,MAC000002,2012-10-12 02:30:00,0,2012-10-12,02:30:00
...,...,...,...,...,...
1222665,MAC005492,2014-02-27 22:00:00,0.182,2014-02-27,22:00:00
1222666,MAC005492,2014-02-27 22:30:00,0.122,2014-02-27,22:30:00
1222667,MAC005492,2014-02-27 23:00:00,0.14,2014-02-27,23:00:00
1222668,MAC005492,2014-02-27 23:30:00,0.192,2014-02-27,23:30:00


In [56]:
# Check the percentage of missing values for each column
missing_data = load.isnull().mean() * 100

print("missing data")
print(missing_data)


missing data
LCLid             0.0
tstp              0.0
energy(kWh/hh)    0.0
date              0.0
time              0.0
dtype: float64


In [57]:
load['energy(kWh/hh)'] = pd.to_numeric(load['energy(kWh/hh)'],  errors='coerce')

In [58]:
# Find the unique dates and times
unique_dates = load['date'].unique()
unique_times = load['time'].unique()
unique_id = load['LCLid'].unique()


In [59]:
unique_times

array([datetime.time(0, 30), datetime.time(1, 0), datetime.time(1, 30),
       datetime.time(2, 0), datetime.time(2, 30), datetime.time(3, 0),
       datetime.time(3, 30), datetime.time(4, 0), datetime.time(4, 30),
       datetime.time(5, 0), datetime.time(5, 30), datetime.time(6, 0),
       datetime.time(6, 30), datetime.time(7, 0), datetime.time(7, 30),
       datetime.time(8, 0), datetime.time(8, 30), datetime.time(9, 0),
       datetime.time(9, 30), datetime.time(10, 0), datetime.time(10, 30),
       datetime.time(11, 30), datetime.time(12, 0), datetime.time(12, 30),
       datetime.time(13, 0), datetime.time(13, 30), datetime.time(14, 0),
       datetime.time(14, 30), datetime.time(15, 0), datetime.time(15, 30),
       datetime.time(16, 0), datetime.time(16, 30), datetime.time(17, 0),
       datetime.time(17, 30), datetime.time(18, 0), datetime.time(18, 30),
       datetime.time(19, 0), datetime.time(19, 30), datetime.time(20, 0),
       datetime.time(20, 30), datetime.time(21, 0)

In [60]:
# The unique times are from 0:00 to 23:30 with interval 30 min
unique_times = unique_times[:48]
# unique_times = np.concatenate([unique_times[19:], unique_times[:19]]) 

In [63]:
unique_times = np.sort(unique_times)
unique_times

array([datetime.time(0, 0), datetime.time(0, 30), datetime.time(1, 0),
       datetime.time(1, 30), datetime.time(2, 0), datetime.time(2, 30),
       datetime.time(3, 0), datetime.time(3, 30), datetime.time(4, 0),
       datetime.time(4, 30), datetime.time(5, 0), datetime.time(5, 30),
       datetime.time(6, 0), datetime.time(6, 30), datetime.time(7, 0),
       datetime.time(7, 30), datetime.time(8, 0), datetime.time(8, 30),
       datetime.time(9, 0), datetime.time(9, 30), datetime.time(10, 0),
       datetime.time(10, 30), datetime.time(11, 0), datetime.time(11, 30),
       datetime.time(12, 0), datetime.time(12, 30), datetime.time(13, 0),
       datetime.time(13, 30), datetime.time(14, 0), datetime.time(14, 30),
       datetime.time(15, 0), datetime.time(15, 30), datetime.time(16, 0),
       datetime.time(16, 30), datetime.time(17, 0), datetime.time(17, 30),
       datetime.time(18, 0), datetime.time(18, 30), datetime.time(19, 0),
       datetime.time(19, 30), datetime.time(20, 0), 

In [66]:
num_dates = len(unique_dates)
num_intervals = len(unique_times)
num_id = len(unique_id)
print("num_id: {}, num_dates: {}, num_intervals per day: {}".format(num_id, num_dates, num_intervals))

num_id: 50, num_dates: 819, num_intervals per day: 48


In [67]:
# Create a mapping of date and time to index
date_to_index = {date: idx for idx, date in enumerate(unique_dates)}
time_to_index = {time: idx for idx, time in enumerate(unique_times)}
dataid_to_index = {dataid: idx for idx, dataid in enumerate(unique_id)}

In [68]:
mean_by_id = list(load.groupby('LCLid')['energy(kWh/hh)'].mean())
mean_by_id = torch.tensor(mean_by_id).view(-1, 1, 1)

In [69]:
# Initialize a new tensor for 'grid' values, missing value replaced by mean
tensor_energy = mean_by_id.repeat(1,num_dates,num_intervals)

# Fill the tensor with 'grid' values
for _, row in load.iterrows():
    if row['time'] in unique_times:
        dataid_index = dataid_to_index[row['LCLid']]
        date_index = date_to_index[row['date']]
        time_index = time_to_index[row['time']]
        tensor_energy[dataid_index, date_index, time_index] = row['energy(kWh/hh)']

In [70]:
torch.save(tensor_energy, "./data/London/energy_{}.pt".format(block))